##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker를 사용한 이미지 분류

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/tutorials/model_maker_image_classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/lite/tutorials/model_maker_image_classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/lite/tutorials/model_maker_image_classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/lite/tutorials/model_maker_image_classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF 허브 모델 보기</a></td>
</table>

[TensorFlow Lite Model Maker 라이브러리](https://www.tensorflow.org/lite/guide/model_maker)를 사용하면 장치 내 ML 애플리케이션용으로 TensorFlow 신경망 모델을 배포할 때 모델을 특정 입력 데이터로 조정하고 변환하는 프로세스가 단순해집니다.

이 노트북은 이 Model Maker 라이브러리를 활용하여 모바일 기기에서 꽃을 분류하기 위해 일반적으로 사용되는 이미지 분류 모델의 적응 및 변환을 설명하는 엔드 투 엔드 예제를 보여줍니다.

## 전제 조건

이 예제를 실행하려면 먼저 Github [repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)에 있는 Model Maker 패키지를 포함하여 여러 필수 패키지를 설치해야 합니다.

In [ ]:
!pip install -q tflite-model-maker

필요한 패키지를 가져옵니다.

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

## 간단한 엔드 투 엔드 예제

### 데이터 경로 가져오기

이 간단한 엔드 투 엔드 예제로 재생할 이미지를 가져오겠습니다. 수백 개의 이미지로 Model Maker를 시작하는 것은 좋지만, 데이터가 많을수록 정확성이 높아집니다.

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

`image_path`를 자신의 이미지 폴더로 바꿀 수 있습니다. colab에 데이터를 업로드하는 경우, 아래 이미지에 빨간색 사각형으로 표시된 왼쪽 사이드 바에서 Upload 버튼을 찾을 수 있습니다. zip 파일을 업로드하고 압축을 풉니다. 루트 파일 경로는 현재 경로입니다.

 <img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_image_classification.png" alt="Upload File"> 

클라우드에 이미지를 업로드하지 않으려면, Github의 [가이드](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)에 따라 로컬에서 라이브러리를 실행할 수 있습니다.

### 예제 실행하기

예제는 아래와 같이 4줄의 코드로 구성되며, 각 줄은 전체 프로세스의 한 단계를 나타냅니다.


1단계. 기기 내 ML 앱과 관련된 입력 데이터를 로드합니다. 훈련 데이터와 테스트 데이터로 분할합니다.

In [ ]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

2단계. TensorFlow 모델을 사용자 정의합니다.

In [ ]:
model = image_classifier.create(train_data)

3단계. 모델을 평가합니다.

In [ ]:
loss, accuracy = model.evaluate(test_data)

4단계. TensorFlow Lite 모델로 내보냅니다.

여기에서는 모델 설명에 대한 표준을 제공하는 [메타데이터](https://www.tensorflow.org/lite/convert/metadata)와 함께 TensorFlow Lite 모델을 내보냅니다. 레이블 파일은 메타데이터에 포함됩니다. 기본 훈련 후 양자화 기술은 이미지 분류 작업에 대한 전체 정수 양자화입니다.

업로드 부분과 같은 왼쪽 사이드 바에서 다운로드하여 사용할 수 있습니다.

In [ ]:
model.export(export_dir='.')

이 간단한 4가지 단계 다음에는 [이미지 분류](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification) 참조 앱과 같은 기기 내 애플리케이션에서 TensorFlow Lite 모델 파일을 추가로 사용할 수 있습니다.

## 세부 프로세스

현재 EfficientNet-Lite * 모델, MobileNetV2, ResNet50과 같은 여러 모델을 이미지 분류를 위한 사전 훈련 모델로 지원합니다. 그러나 몇 줄의 코드만으로 새로운 사전 훈련 모델을 이 라이브러리에 자유롭게 추가할 수 있습니다.

다음은 자세한 내용을 보여주기 위해 엔드 투 엔드 예제를 단계별로 안내합니다.

### 1단계: 기기 내 ML 앱과 관련된 입력 데이터 로드하기

꽃 데이터세트에는 5개 클래스에 속하는 3670개의 이미지가 포함되어 있습니다. 데이터세트의 아카이브 버전을 다운로드하고 압축을 풉니다.

데이터세트에는 다음과 같은 디렉토리 구조가 있습니다.

<pre>&lt;b&gt;flower_photos&lt;/b&gt;
|__ &lt;b&gt;daisy&lt;/b&gt;
    |______ 100080576_f52e8ee070_n.jpg
    |______ 14167534527_781ceb1b7a_n.jpg
    |______ ...
|__ &lt;b&gt;dandelion&lt;/b&gt;
    |______ 10043234166_e6dd915111_n.jpg
    |______ 1426682852_e62169221f_m.jpg
    |______ ...
|__ &lt;b&gt;roses&lt;/b&gt;
    |______ 102501987_3cdb8e5394_n.jpg
    |______ 14982802401_a3dfb22afb.jpg
    |______ ...
|__ &lt;b&gt;sunflowers&lt;/b&gt;
    |______ 12471791574_bb1be83df4.jpg
    |______ 15122112402_cafa41934f.jpg
    |______ ...
|__ &lt;b&gt;tulips&lt;/b&gt;
    |______ 13976522214_ccec508fe7.jpg
    |______ 14487943607_651e8062a1_m.jpg
    |______ ...
</pre>

In [ ]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

`DataLoader` 클래스를 사용하여 데이터를 로드합니다.

`from_folder()` 메서드의 경우, 폴더에서 데이터를 로드할 수 있습니다. 같은 클래스의 이미지 데이터는 같은 하위 디렉토리에 있고 하위 폴더 이름이 클래스 이름이라고 가정합니다. 현재 JPEG로 인코딩된 이미지와 PNG로 인코딩된 이미지가 지원됩니다.

In [ ]:
data = DataLoader.from_folder(image_path)

훈련 데이터(80%), 검증 데이터(10%, 선택 사항) 및 테스트 데이터(10%)로 분할합니다.

In [ ]:
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

레이블이 있는 25개의 이미지 예시를 보여줍니다.

In [ ]:
plt.figure(figsize=(10,10))
for i, (image, label) in enumerate(data.gen_dataset().unbatch().take(25)):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)
  plt.xlabel(data.index_to_label[label.numpy()])
plt.show()

### 2단계: TensorFlow 모델 사용자 정의하기

로드된 데이터를 기반으로 사용자 정의 이미지 분류자 모델을 만듭니다. 기본 모델은 EfficientNet-Lite0입니다.


In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data)

자세한 모델 구조를 살펴봅니다

In [ ]:
model.summary()

### 3단계: 맞춤형 모델 평가하기

모델의 결과를 평가하고 모델의 손실과 정확성을 얻습니다.

In [ ]:
loss, accuracy = model.evaluate(test_data)

예측 결과를 100개의 테스트 이미지로 플롯할 수 있습니다. 빨간색으로 예측된 레이블은 잘못된 예측 결과이고 다른 레이블은 정확합니다.

In [ ]:
# A helper function that returns 'red'/'black' depending on if its two input
# parameter matches or not.
def get_label_color(val1, val2):
  if val1 == val2:
    return 'black'
  else:
    return 'red'

# Then plot 100 test images and their predicted labels.
# If a prediction result is different from the label provided label in "test"
# dataset, we will highlight it in red color.
plt.figure(figsize=(20, 20))
predicts = model.predict_top_k(test_data)
for i, (image, label) in enumerate(test_data.gen_dataset().unbatch().take(100)):
  ax = plt.subplot(10, 10, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image.numpy(), cmap=plt.cm.gray)

  predict_label = predicts[i][0][0]
  color = get_label_color(predict_label,
                          test_data.index_to_label[label.numpy()])
  ax.xaxis.label.set_color(color)
  plt.xlabel('Predicted: %s' % predict_label)
plt.show()

정확성이 앱 요구 사항을 충족하지 않는 경우, [고급 사용법](#scrollTo=zNDBP2qA54aK)을 참조하여 더 큰 모델로 변경하거나 재훈련 매개변수를 조정하는 방법 등과 같은 대안을 탐색할 수 있습니다.

### 4단계: TensorFlow Lite 모델로 내보내기

나중에 장치 내 ML 애플리케이션에서 사용할 수 있도록 훈련된 모델을 [메타데이터](https://www.tensorflow.org/lite/convert/metadata)가 포함된 TensorFlow Lite 모델 형식으로 변환합니다. 레이블 파일과 어휘 파일은 메타데이터에 포함됩니다. 기본 TFLite 파일 이름은 `model.tflite`입니다.

많은 기기 내 ML 애플리케이션에서 모델 크기는 중요한 요소입니다. 따라서 모델을 더 작게 만들고 잠재적으로 더 빠르게 실행하려면 양자화를 적용하는 것이 좋습니다. 기본 훈련 후 양자화 기술은 이미지 분류 작업에 대한 전체 정수 양자화입니다.

In [ ]:
model.export(export_dir='.')

TensorFlow Lite 모델을 모바일 앱에 통합하는 방법에 대한 자세한 내용은 [예제 애플리케이션 및 이미지 분류 가이드](https://www.tensorflow.org/lite/models/image_classification/overview#example_applications_and_guides)를 참조하세요.

이 모델은 [TensorFlow Lite 작업 라이브러리](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview)의 [ImageClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier)를 사용하여 Android 또는 iOS 앱에 통합할 수 있습니다.

허용되는 내보내기 형식은 다음 중 하나 또는 목록일 수 있습니다.

- `ExportFormat.TFLITE`
- `ExportFormat.LABEL`
- `ExportFormat.SAVED_MODEL`

기본적으로, 메타 데이터와 함께 TensorFlow Lite 모델만 내보냅니다. 다른 파일을 선택적으로 내보낼 수도 있습니다. 예를 들어, 다음과 같이 레이블 파일만 내보냅니다.

In [ ]:
model.export(export_dir='.', export_format=ExportFormat.LABEL)

`evaluate_tflite` 메서드로 Tflite 모델을 평가할 수도 있습니다.

In [ ]:
model.evaluate_tflite('model.tflite', test_data)

## 고급 사용법

`create` 함수는 이 라이브러리의 중요한 부분입니다. [튜토리얼](https://www.tensorflow.org/tutorials/images/transfer_learning)과 유사한 사전 훈련 모델과 함께 전이 학습을 사용합니다.

`create` 함수에는 다음 단계가 포함됩니다.

1. 매개변수 `validation_ratio` 및 `test_ratio`에 따라 데이터를 훈련, 검증, 테스트 데이터로 분할합니다. `validation_ratio` 및 `test_ratio`의 기본값은 `0.1` 및 `0.1`입니다.
2. TensorFlow Hub에서 [이미지 특성 벡터](https://www.tensorflow.org/hub/common_signatures/images#image_feature_vector)를 기본 모델로 다운로드합니다. 사전 훈련된 기본 모델은 EfficientNet-Lite0입니다.
3. 헤드 레이어와 사전 훈련된 모델 사이에 `dropout_rate`를 가진 Dropout 레이어가 있는 분류자 헤드를 추가합니다. 기본 `dropout_rate`는 TensorFlow Hub에서 <a>make_image_classifier_lib</a>의 기본 <code>dropout_rate</code> 값입니다.
4. 원시 입력 데이터를 전처리합니다. 현재는 각 이미지 픽셀의 값을 모델 입력 배율로 정규화하고 모델 입력 크기에 맞게 크기를 조정하는 등 전처리 단계가 있습니다. EfficientNet-Lite0의 입력 배율은 `[0, 1]`이고 입력 이미지 크기는 `[224, 224, 3]`입니다.
5. 분류자 모델에 데이터를 입력합니다. 기본적으로 훈련 epoch, 배치 크기, 학습률, 모멘텀과 같은 훈련 매개변수는 TensorFlow Hub [make_image_classifier_lib](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/tools/make_image_classifier/make_image_classifier_lib.py#L55)의 기본값입니다. 분류자 헤드만 훈련됩니다.

이 섹션에서는 다른 이미지 분류 모델로 전환하기, 훈련 하이퍼 매개변수 변경하기 등 여러 고급 주제에 대해 설명합니다.


## TensorFlow Lite 모델에서 훈련 후 양자화 사용자 지정하기


[훈련 후 양자화](https://www.tensorflow.org/lite/performance/post_training_quantization)는 모델 정확도를 약간만 떨어트리면서 모델 크기와 추론 지연 시간을 줄이는 동시에 CPU 및 하드웨어 가속기의 추론 속도도 개선할 수 있는 변환 기술입니다. 따라서 모델을 최적화하는 데 널리 사용됩니다.


Model Maker 라이브러리는 모델을 내보낼 때 기본 훈련 후 양자화 기술을 적용합니다. 훈련 후 양자화를 사용자 지정하려는 경우 Model Maker는 [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig)를 사용하여 여러 훈련 후 양자화 옵션도 지원합니다. float16 양자화를 예로 들어보겠습니다. 먼저 양자화 구성을 정의합니다.

In [ ]:
config = QuantizationConfig.for_float16()

그런 다음 이러한 구성을 가진 TensorFlow Lite 모델을 내보냅니다.

In [ ]:
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)

Colab에서, 위에서 언급한 업로드 부분과 동일하게 왼쪽 사이드 바에서 `model_quant.tflite`라는 모델을 다운로드할 수 있습니다.

## 모델 변경하기


### 이 라이브러리에서 지원되는 모델로 변경합니다.

이 라이브러리는 현재 EfficientNet-Lite 모델, MobileNetV2, ResNet50을 지원합니다. [EfficientNet-Lite](https://github.com/tensorflow/tpu/tree/master/models/official/efficientnet/lite)는 최첨단 정확성을 달성하며 Edge 기기에 적합한 이미지 분류 모델 제품군입니다. 기본 모델은 EfficientNet-Lite0입니다.

`create` 메서드에서 `model_spec` 매개변수를 MobileNetV2 모델 사양으로 설정하기만 하면 모델을 MobileNetV2로 전환할 수 있습니다.

In [ ]:
model = image_classifier.create(train_data, model_spec=model_spec.get('mobilenet_v2'), validation_data=validation_data)

새로 재훈련된 MobileNetV2 모델을 평가하여 테스트 데이터의 정확성과 손실을 확인합니다.

In [ ]:
loss, accuracy = model.evaluate(test_data)

### TensorFlow Hub 모델로 변경하기

또한, 이미지를 입력하고 TensorFlow Hub 형식으로 특성 벡터를 출력하는 다른 새 모델로 전환할 수도 있습니다.

[Inception V3](https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1) 모델의 예로, [image_classifier.ModelSpec](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/image_classifier/ModelSpec)의 객체이며 Inception V3 모델의 사양을 포함하는 `inception_v3_spec`을 정의할 수 있습니다.

모델 이름 `name`과 TensorFlow Hub 모델의 URL `uri`를 지정해야 합니다. 반면에 `input_image_shape`의 기본값은 `[224, 224]`입니다. Inception V3 모델의 경우, `[299, 299]`로 변경해야 합니다.

In [ ]:
inception_v3_spec = image_classifier.ModelSpec(
    uri='https://tfhub.dev/google/imagenet/inception_v3/feature_vector/1')
inception_v3_spec.input_image_shape = [299, 299]

그런 다음 `create` 메서드에서 매개변수 `model_spec`을 `inception_v3_spec`으로 설정하여 Inception V3 모델을 재훈련할 수 있습니다.

나머지 단계는 정확히 같고, 최종적으로 맞춤형 InceptionV3 TensorFlow Lite 모델을 얻을 수 있습니다.

### 사용자 정의 모델 변경하기

TensorFlow Hub에 없는 사용자 정의 모델을 사용하려면, TensorFlow Hub에서 [ModelSpec](https://www.tensorflow.org/hub/api_docs/python/hub/ModuleSpec)을 생성하고 내보내야 합니다.

그런 다음 위의 프로세스와 같이 `ModelSpec` 객체를 정의하기 시작합니다.

## 훈련 하이퍼 매개변수 변경하기

모델 정확도에 영향을 줄 수 있는 `epochs`, `dropout_rate` 및 `batch_size`와 같은 훈련 하이퍼파라미터를 변경할 수도 있습니다. 조정할 수 있는 모델 매개변수는 다음과 같습니다.

- `epochs`: 정확성이 수렴될 때까지 더 많은 epoch에서 더 나은 정확성을 달성할 수 있지만, 너무 많은 epoch에서 훈련하면 과대적합으로 이어질 수 있습니다.
- `dropout_rate`: 과대적합을 피합니다. 기본값은 None입니다.
- `batch_size`: 하나의 훈련 단계에서 사용할 샘플의 수입니다. 기본값은 None입니다.
- `validation_data`: 검증 데이터입니다. 기본값은 없음입니다.
- `train_whole_model`: true인 경우 Hub 모듈은 상단의 분류 레이어와 함께 훈련됩니다. 그렇지 않으면 최상위 분류 레이어만 훈련합니다. 기본값은 None입니다.
- `learning_rate`: 기본 학습률, 기본값은 None입니다.
- `momentum`: 옵티마이저로 전달되는 Python float, `use_hub_library`가 ​​True 인 경우에만 사용됩니다. 기본값은 None입니다.
- `shuffle`: 부울, 데이터를 셔플해야 하는지 여부를 나타냅니다. 기본값은 False입니다.
- `use_augmentation`: 부울, 전처리를 위해 데이터 증대를 사용합니다. 기본값은 False입니다.
- `use_hub_library`: 부울, tensorflow hub에서 `make_image_classifier_lib`을 사용하여 모델을 다시 훈련합니다. 이 훈련 파이프라인은 카테고리가 많은 복잡한 데이터세트에서 더 나은 성능을 얻을 수 있습니다. 기본값은 True입니다.
- `warmup_steps`: 학습률에 대한 워밍업 스케줄에 대한 워밍업 단계의 수입니다. None이면, 두 epoch에서 총 훈련 단계로 기본 warmup_steps가 사용됩니다. `use_hub_library`가 ​​False인 경우에만 사용됩니다. 기본값은 None입니다.
- `model_dir`: 선택 사항, 모델 체크포인트 파일의 위치, `use_hub_library`가 ​​False인 경우에만 사용됩니다. 기본값은 None입니다.

<code>epochs</code>와 같이 기본적으로 None인 매개변수는 TensorFlow Hub 라이브러리 또는 <a>train_image_classifier_lib</a>에서 <a>make_image_classifier_lib</a>의 구체적인 기본 매개변수를 가져옵니다.

예를 들어, 더 많은 epoch에서 훈련할 수 있습니다.


In [ ]:
model = image_classifier.create(train_data, validation_data=validation_data, epochs=10)

10개의 훈련 epoch에서 새로 재훈련된 모델을 평가합니다.

In [ ]:
loss, accuracy = model.evaluate(test_data)

# 더 읽어보기

기술적인 세부 사항을 알아보려면 [이미지 분류](https://www.tensorflow.org/lite/examples/image_classification/overview) 예제를 읽어볼 수 있습니다. 자세한 내용은 다음을 참조하세요.

- TensorFlow Lite Model Maker [가이드](https://www.tensorflow.org/lite/guide/model_maker) 및 [API 참조](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)
- 작업 라이브러리: 배포용 [ImageClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/image_classifier)
- 엔드 투 엔드 참조 앱: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/ios) 및 [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/raspberry_pi)
